In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from idaes.surrogate.pysmo.polynomial_regression import PolynomialRegression
from idaes.surrogate.pysmo.radial_basis_function import RadialBasisFunctions
from idaes.surrogate.pysmo.kriging import KrigingModel
import json

In [2]:
import pysmo_surrogate as ps

In [3]:
training_data = {'x1': [1, 2, 3, 4, 5], 'x2': [5, 6, 7, 8, 9], 'z1': [10, 20, 30, 40, 50], 'z2': [6, 8, 10, 12, 14]}
training_data = pd.DataFrame(training_data)
validation_data = {'x1': [1, 2, 3, 4], 'x2': [5, 6, 7, 8], 'z1': [10, 20, 30, 40], 'z2': [6, 8, 10, 12]}#{'x1': [2.5], 'x2': [6.5], 'z1': [25], 'z2': [9]}
validation_data = pd.DataFrame(validation_data)
input_labels = ["x1", "x2"]
output_labels = ["z1", "z2"]
bnds = {"x1": (0, 5), "x2": (0, 10)}

In [4]:
pysmo_trainer = ps.PysmoPolyTrainer(
    input_labels=input_labels,
    output_labels=output_labels,
    # input_bounds=bnds,
    training_dataframe=training_data,
    validation_dataframe=validation_data,
    maximum_polynomial_order = 1, 
    multinomials=False,
    number_of_crossvalidations=5,
    solution_method='pyomo')

a = pysmo_trainer.train_surrogate()


===========================Polynomial Regression===============================================


No iterations will be run.
Parameter estimation method:  pyomo 

max_fraction_training_samples set at  0.5
Number of adaptive samples (no_adaptive_samples) set at  4
Maximum number of iterations (Max_iter) set at:  0

Initial surrogate model is of order 1  with a cross-val error of 0.000000
Initial Regression Model Performance:
Order:  1  / MAE: 0.000000  / MSE: 0.000000  / R^2: 1.000000  / Adjusted R^2: 1.000000

Polynomial regression generates a good surrogate model for the input data.

-------------------------------------------------
-------------------------------------------------
Best solution found:  
Order:  1  / MAE: 0.000000  / MSE: 0.000000  / R_sq: 1.000000  / Adjusted R^2: 1.000000

------------------------------------------------------------
The final coefficients of the regression terms are: 

k               | -8.9
(x_ 1 )^ 1      | 7.775
(x_ 2 )^ 1      | 2.225

Results 

In [5]:
p2 = ps.PysmoSurrogate(a, input_labels, output_labels)

In [6]:
zv = p2.save()

In [7]:
a1 = json.loads(zv)

In [8]:
from pyomo.core.base.param import IndexedParam
from pyomo.environ import Param

def str_conv_back(xyz, p):
    list_idx_vars = [p._data[i].local_name for i in p._data.keys()]
    list_vars = ['p["'+str(i)+'"]' for i in p.keys()]
    pyomo_vars_expr = xyz
    for i in range(0, len(list_idx_vars)):
        pyomo_vars_expr = [var_name.replace(list_idx_vars[i], list_vars[i]) for var_name in pyomo_vars_expr]
    return pyomo_vars_expr
    # return [eval(r, {}, {"p":p}) for r in pyomo_vars_expr]

def str_deserialize(v):
    return v

def list_deserialize(v):
    return v

def numpy_deserialize(v):
    return np.array(v)

def pandas_deserialize(v):
    return pd.read_json(v, orient='index')

switcher = {
    'numpy': numpy_deserialize,
    'pandas':pandas_deserialize,
    'str': str_deserialize,
    'list': list_deserialize
    }

class PolyUnserializer(PolynomialRegression):
    def __init__(self, dictionary, dictionary_map):
        for k, v in dictionary.items():
            if k not in ['feature_list',"extra_terms_feature_vector", "additional_term_expressions"]: 
                setattr(self, k, switcher.get(dictionary_map[k])(v))
            else:
                pass
            
        p = Param(self.regression_data_columns, mutable=True, initialize=0)
        p.index_set().construct()
        p.construct()
        setattr(self, 'feature_list', p)
        self.extra_terms_feature_vector = list(self.feature_list[i] for i in self.regression_data_columns)
        list_terms = str_conv_back(dictionary['additional_term_expressions'], p)
        setattr(self, 'additional_term_expressions', [eval(m, {}, {"p":p}) for m in list_terms])

class RbfUnserializer(RadialBasisFunctions):
    def __init__(self, dictionary, dictionary_map):
        for k, v in dictionary.items():
                setattr(self, k, switcher.get(dictionary_map[k])(v))


class KrigingUnserializer(KrigingModel):
    def __init__(self, dictionary, dictionary_map):
        for k, v in dictionary.items():
                setattr(self, k, switcher.get(dictionary_map[k])(v))



In [9]:
class PysmoDeserializer():
  def __init__(self, dt_in):
    self._results = {}
    self._results['model type'] = dt_in['surrogate_type']
    self._results['pysmo_results'] = {}
    output_list = list(dt_in['model_encoding'])
    for i in range(0, len(output_list)): 
      if self._results['model type'] == 'poly':
        self._results['pysmo_results'][output_list[i]] = PolyUnserializer(dt_in['model_encoding'][output_list[i]]['attr'], dt_in['model_encoding'][output_list[i]]['map'])
      elif self._results['model type'] in ['linear rbf', 'cubic rbf', 'gaussian rbf', 'mq rbf', 'imq rbf', 'spline rbf']:
        self._results['pysmo_results'][output_list[i]] = RbfUnserializer(dt_in['model_encoding'][output_list[i]]['attr'], dt_in['model_encoding'][output_list[i]]['map'])
      elif self._results['model type'] == 'kriging':
        self._results['pysmo_results'][output_list[i]] = KrigingUnserializer(dt_in['model_encoding'][output_list[i]]['attr'], dt_in['model_encoding'][output_list[i]]['map'])
    
bb = PysmoDeserializer(a1)

In [10]:
input_labels = a1["input_labels"]
output_labels = a1["output_labels"]

qq = ps.PysmoSurrogate(surrogate_expressions=bb,
                        input_labels=input_labels,
                        output_labels=output_labels)

In [11]:
from idaes.surrogate.surrogate_block import SurrogateBlock

blk4 = SurrogateBlock(concrete=True)
blk4.build_model(qq)
blk4.pprint()

3 Set Declarations
    inputs_set : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'x1', 'x2'}
    outputs_set : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'z1', 'z2'}
    pysmo_constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'z1', 'z2'}

2 Var Declarations
    inputs : Size=2, Index=inputs_set
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         x1 :  None :     0 :  None : False : False :  Reals
         x2 :  None :     0 :  None : False : False :  Reals
    outputs : Size=2, Index=outputs_set
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         z1 :  None :     0 :  None : False : False :  Reals
         z2 :  None :     0 :  None : False : False :  Reals

1 Constraint Declarations
    pysmo_const

In [12]:
qq.evaluate_surrogate(validation_data[['x1', 'x2']])

,z1,z2
0,10.0,6.0
1,20.0,8.0
2,30.0,10.0
3,40.0,12.0


In [13]:
pysmo_trainer = ps.PysmoRBFTrainer(
    input_labels=input_labels,
    output_labels=output_labels,
    # input_bounds=bnds,
    training_dataframe=training_data,
    validation_dataframe=validation_data,
    solution_method = 'bfgs',
    regularization=False,
    basis_function = 'cubic')

a = pysmo_trainer.train_surrogate()


p22 = ps.PysmoSurrogate(a, input_labels, output_labels)
# Test surrogate evaluation
b = p22.evaluate_surrogate(validation_data[['x1', 'x2']])
zv2 = p22.save()



Parameter estimation method:  bfgs
Basis function:  cubic
Regularization done:  False
0    |     0    |     2.2991910097373442    |     133.6096644614146    |     2.96673051595008e-14    |     133.6096644614146    |     2.96673051595008e-14

Results saved in  z1_solution.pickle

Model for output z1 trained successfully.



Parameter estimation method:  bfgs
Basis function:  cubic
Regularization done:  False
0    |     0    |     2.2991910097373442    |     133.6096644614146    |     2.96673051595008e-14    |     133.6096644614146    |     2.96673051595008e-14

Results saved in  z2_solution.pickle

Model for output z2 trained successfully.



In [14]:
a2 = json.loads(zv2)

In [15]:
a2

{'model_encoding': {'z1': {'attr': {'x_data_columns': ['x1', 'x2'],
    'x_data': [[0.0, 0.0], [0.25, 0.25], [0.5, 0.5], [0.75, 0.75], [1.0, 1.0]],
    'centres': [[0.0, 0.0],
     [0.25, 0.25],
     [0.5, 0.5],
     [0.75, 0.75],
     [1.0, 1.0]],
    'basis_function': 'cubic',
    'weights': [[1.317201961295672],
     [-2.5352365081566393],
     [1.0606601717797937],
     [-1.7074041789626262],
     [0.6273416869673308]],
    'sigma': 0.0,
    'regularization_parameter': 0.0,
    'rmse': 5.563576319889857e-17,
    'R2': 1.0,
    'x_data_min': [[1, 5]],
    'x_data_max': [[5, 9]],
    'y_data_min': [10],
    'y_data_max': [50]},
   'map': {'x_data_columns': 'list',
    'x_data': 'numpy',
    'centres': 'numpy',
    'basis_function': 'str',
    'weights': 'numpy',
    'sigma': 'str',
    'regularization_parameter': 'str',
    'rmse': 'str',
    'R2': 'str',
    'x_data_min': 'numpy',
    'x_data_max': 'numpy',
    'y_data_min': 'numpy',
    'y_data_max': 'numpy'}},
  'z2': {'attr': {'x

In [16]:
cc = PysmoDeserializer(a2)

In [17]:
input_labels = a2["input_labels"]
output_labels = a2["output_labels"]

rr = ps.PysmoSurrogate(surrogate_expressions=cc,
                        input_labels=input_labels,
                        output_labels=output_labels)

In [18]:
rr._surrogate_expressions.__dict__

{'_results': {'model type': 'cubic rbf',
  'pysmo_results': {'z1': 
Results of radial basis function run:

Basis function type               : cubic
Shape parameter                    : 0.0
Regularization parameter           : 0.0
Number of terms in RBF model       : 6

RBF Expression:
--------------------------

10 + 40*(1.317201961295672*((((IndexedParam[x1] -1)/4)**2 + ((IndexedParam[x2] -5)/4)**2)**0.5)**3 -2.5352365081566393*((((IndexedParam[x1] -1)/4 -0.25)**2 + ((IndexedParam[x2] -5)/4 -0.25)**2)**0.5)**3 + 1.0606601717797937*((((IndexedParam[x1] -1)/4 -0.5)**2 + ((IndexedParam[x2] -5)/4 -0.5)**2)**0.5)**3 -1.7074041789626262*((((IndexedParam[x1] -1)/4 -0.75)**2 + ((IndexedParam[x2] -5)/4 -0.75)**2)**0.5)**3 + 0.6273416869673308*((((IndexedParam[x1] -1)/4 -1.0)**2 + ((IndexedParam[x2] -5)/4 -1.0)**2)**0.5)**3)
--------------------------

Model training errors:
-----------------------
Mean Squared Error (MSE)         : 3.095338146723916e-33
Root Mean Squared Error (RMSE)   : 5.56

In [19]:
rr.evaluate_surrogate(validation_data[['x1', 'x2']])

,z1,z2
0,10.0,6.0
1,20.0,8.0
2,30.0,10.0
3,40.0,12.0


In [20]:
pysmo_trainer = ps.PysmoKrigingTrainer(
    input_labels=input_labels,
    output_labels=output_labels,
    # input_bounds=bnds,
    training_dataframe=training_data,
    validation_dataframe=validation_data)

a = pysmo_trainer.train_surrogate()


p3 = ps.PysmoSurrogate(a, input_labels, output_labels)
# Test surrogate evaluation
b = p3.evaluate_surrogate(validation_data[['x1', 'x2']])
# Test block generation
zv3 = p3.save()


Optimizing kriging parameters using L-BFGS-B algorithm...

Final results
Theta: [0.001 0.001] 
Mean: [[30.00000079]] 
Regularization parameter: 1e-10

Results saved in  z1_solution.pickle

Model for output z1 trained successfully.


Optimizing kriging parameters using L-BFGS-B algorithm...

Final results
Theta: [0.001 0.001] 
Mean: [[10.00000016]] 
Regularization parameter: 1e-10

Results saved in  z2_solution.pickle

Model for output z2 trained successfully.



In [21]:
a3 = json.loads(zv3)

In [22]:
a3

{'model_encoding': {'z1': {'attr': {'x_data_columns': ['x1', 'x2'],
    'x_data': [[1, 5], [2, 6], [3, 7], [4, 8], [5, 9]],
    'x_data_min': [[1, 5]],
    'x_data_max': [[5, 9]],
    'x_data_scaled': [[0.0, 0.0],
     [0.25, 0.25],
     [0.5, 0.5],
     [0.75, 0.75],
     [1.0, 1.0]],
    'optimal_weights': [0.001, 0.001],
    'optimal_p': 2,
    'optimal_mean': [[30.000000794640247]],
    'optimal_variance': [[112787.69806671143]],
    'regularization_parameter': 1e-10,
    'optimal_covariance_matrix': [[1.0000000001,
      0.9998750078121745,
      0.9995001249791693,
      0.998875632575262,
      0.9980019986673331],
     [0.9998750078121745,
      1.0000000001,
      0.9998750078121745,
      0.9995001249791693,
      0.998875632575262],
     [0.9995001249791693,
      0.9998750078121745,
      1.0000000001,
      0.9998750078121745,
      0.9995001249791693],
     [0.998875632575262,
      0.9995001249791693,
      0.9998750078121745,
      1.0000000001,
      0.9998750078121745

In [23]:
dd = PysmoDeserializer(a3)

In [24]:
input_labels = a3["input_labels"]
output_labels = a3["output_labels"]

ss = ps.PysmoSurrogate(surrogate_expressions=dd,
                        input_labels=input_labels,
                        output_labels=output_labels)

In [25]:
ss._surrogate_expressions.__dict__

{'_results': {'model type': 'kriging',
  'pysmo_results': {'z1': 
Results of Kriging run:

Kriging mean                     : [[30.00000079]]
Kriging variance                 : [[112787.69806671]]
Kriging weights                  : [0.001 0.001]
Regularization parameter         : 1e-10
Number of terms in Kriging model : 6

Kriging Expression:
--------------------

30.000000794640247 + (-10919704.75186791*exp(- (0.001*((IndexedParam[x1] -1)/4)**2 + 0.001*((IndexedParam[x2] -5)/4)**2)) + 21811213.041695453*exp(- (0.001*((IndexedParam[x1] -1)/4 -0.25)**2 + 0.001*((IndexedParam[x2] -5)/4 -0.25)**2)) -1.3868303361060013*exp(- (0.001*((IndexedParam[x1] -1)/4 -0.5)**2 + 0.001*((IndexedParam[x2] -5)/4 -0.5)**2)) -21811211.192968994*exp(- (0.001*((IndexedParam[x1] -1)/4 -0.75)**2 + 0.001*((IndexedParam[x2] -5)/4 -0.75)**2)) + 10919704.289977863*exp(- (0.001*((IndexedParam[x1] -1)/4 -1.0)**2 + 0.001*((IndexedParam[x2] -5)/4 -1.0)**2)))
--------------------------

Model training errors:
---------

In [26]:
ss.evaluate_surrogate(validation_data[['x1', 'x2']])

,z1,z2
0,10.001106,6.000221
1,19.997830,7.999566
2,30.000012,10.000002
3,40.002189,12.000438
